In [67]:
import numpy as np
from tqdm import tqdm

In [5]:
std_ = np.ones((10,))
mean_ = []
def reward(i:int,std_:np.array,mean_:list)->float:
    return np.random.normal(loc = mean_[i], scale=std_[i])


In [64]:
def input_checker(func):
    def wrapper(*args, **kwargs):
        assert isinstance(kwargs['mean_'], (np.ndarray,list)), "invalid format of 'mean_'"
        assert isinstance(kwargs['std_'], (np.ndarray,list)), "invalid format of 'std_'"
        assert isinstance(kwargs['k'],int), 'k needs to be an integer'
        assert kwargs['k'] > 1, 'the valud of k should be more than 1, {0} is given!'.format(kwargs['k'])
        assert kwargs['k'] == len(kwargs['mean_'])==len(kwargs['std_']),'The length of std_, mean, and k are not consistent'
        assert isinstance(kwargs['epsilon'],float), 'The value of epsilon needs to be a float'
        assert (kwargs['epsilon']>=0) & (kwargs['epsilon']<=1.0), 'The value of epsilion needs to be between 0 and 1.0'
        assert isinstance(kwargs['n'],int), 'the format of n is incorrect'
        return func(*args, **kwargs)
    return wrapper

# ===============================================

@input_checker
class Epsilon_Greedy:
    def __init__(self,
                 std_:np.ndarray = None,
                 mean_:np.ndarray = None,
                 epsilon:float = None,
                 k:int = None,
                 n:int = None) -> None:
        self.std_ = std_
        self.mean_ = mean_
        self.k = k
        self.epsilon = epsilon
        self.n=n
        self.Q = np.zeros((self.k))
        self.A = np.random.choice(range(self.k - 1))
        self.Q_hist = np.zeros((self.k,n))
        self.A_hist = np.zeros((self.n,))*np.nan
        self.R = 0
        self.action_counter = np.zeros((self.k,))
    
    def reward(self, ind:int)->float:
        return np.random.normal(loc = self.mean_[ind], scale=self.std_[ind])
    
    def run(self):
        for i in tqdm(range(self.n)):
            if np.random.rand() <= self.epsilon or i == 0:
                self.A = np.random.choice(range(self.k - 1))
            else:
                self.A = self.Q.argmax()
            self.action_counter[self.A] += 1
            self.R = self.reward(ind=self.A)
            self.Q[self.A] = self.Q[self.A] + (1/self.action_counter[self.A]) * (self.R - self.Q[self.A])
            self.Q_hist[:, i:i+1] = self.Q[:,None]
            self.A_hist[i] = self.A
        return self.Q_hist, self.A_hist, 

In [66]:
mdl = Epsilon_Greedy(mean_=[2,3,5],std_=[1,1,1],epsilon=0.1, k=3,n=1000)

0.4582480142391715

In [63]:
np.random.choice(range(2))

0